In [ ]:
import sys
import cv2
import mediapipe as mp
import math
import pyttsx3
import time
import os
from datetime import datetime
from docx import Document
from docx.shared import Inches

from PyQt5.QtWidgets import (QApplication, QMainWindow, QLabel, QLineEdit, QPushButton,
                             QVBoxLayout, QWidget, QMessageBox)
from PyQt5.QtCore import QTimer, Qt
from PyQt5.QtGui import QImage, QPixmap
from openai import OpenAI
import speech_recognition as sr

class PoseApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("姿势检测")
        self.subject_id = ""
        
        # UI控件：标签、输入框和按钮
        self.input_label = QLabel("请输入受试者编号:")
        self.subject_input = QLineEdit()
        self.start_button = QPushButton("开始检测")
        self.start_button.clicked.connect(self.start_detection)
        
        # 用于显示摄像头画面的标签
        self.video_label = QLabel()
        self.video_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        
        layout = QVBoxLayout()
        layout.addWidget(self.input_label)
        layout.addWidget(self.subject_input)
        layout.addWidget(self.start_button)
        layout.addWidget(self.video_label)
        
        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)
        
        # 初始化 mediapipe 和 pyttsx3
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose()
        self.mp_drawing = mp.solutions.drawing_utils
        self.engine = pyttsx3.init()
        
        # 摄像头和计时器
        self.cap = None
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_frame)
        
        # 用于逻辑判断的变量
        self.last_time = time.time()
        self.hip_angle_history = []
        self.leg_raised = False

    def speak(self, message):
        # 语音播报并打印消息
        print(message)
        self.engine.say(message)
        self.engine.runAndWait()

    def calculate_angle(self, a, b, c):
        # 计算三点之间的夹角
        ab = [b[0] - a[0], b[1] - a[1]]
        bc = [c[0] - b[0], c[1] - b[1]]
        dot_product = ab[0] * bc[0] + ab[1] * bc[1]
        cross_product = ab[0] * bc[1] - ab[1] * bc[0]
        angle = math.degrees(math.atan2(abs(cross_product), dot_product))
        return angle

    def ask_question(self, question):
        # 使用消息对话框进行简单的“是/否”提问
        msg_box = QMessageBox(self)
        msg_box.setWindowTitle("提问")
        msg_box.setText(question + "\n点击 [是] 或 [否]")
        yes_button = msg_box.addButton("是", QMessageBox.ButtonRole.YesRole)
        no_button = msg_box.addButton("否", QMessageBox.ButtonRole.NoRole)
        msg_box.exec()
        if msg_box.clickedButton() == yes_button:
            return "是"
        else:
            return "否"

    def generate_report_with_image(self, image_path, report_content, report_path):
        """
        生成包含照片和报告内容的 Word 文档
        """
        doc = Document()
        doc.add_heading('姿势检测报告', 0)
        doc.add_paragraph(f"报告生成时间：{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        # # 创建包含症状和描述的表格
        # table = doc.add_table(rows=1, cols=2)
        # table.style = 'Table Grid'
        # hdr_cells = table.rows[0].cells
        # hdr_cells[0].text = '症状'
        # hdr_cells[1].text = '描述'
        # for cell in hdr_cells:
        #     cell.width = Inches(3)
        # # 将大模型返回的报告内容按“；”分隔后添加到表格中
        # symptoms = report_content.split('；')
        # for symptom in symptoms:
        #     row_cells = table.add_row().cells
        #     symptom_parts = symptom.split('：')
        #     if len(symptom_parts) == 2:
        #         row_cells[0].text = symptom_parts[0].strip()
        #         row_cells[1].text = symptom_parts[1].strip()

        doc.add_paragraph(report_content)
        
        doc.add_paragraph("照片：")
        doc.add_picture(image_path, width=Inches(3))
        doc.save(report_path)

    def start_detection(self):
        # 获取受试者编号，并启动摄像头检测
        self.subject_id = self.subject_input.text().strip()
        if not self.subject_id:
            QMessageBox.warning(self, "警告", "请输入受试者编号!")
            return
        # camera
        # self.cap = cv2.VideoCapture(1)
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            QMessageBox.critical(self, "错误", "无法打开摄像头!")
            return
        self.timer.start(30)  # 每30毫秒更新一次画面
        self.start_button.setEnabled(False)
        self.subject_input.setEnabled(False)

    def listen(self,prompt):
        print(prompt)  
        r = sr.Recognizer()
        with sr.Microphone() as source:
            print("请开始说话...")
            audio = r.listen(source)
        try:
            text = r.recognize_google(audio, language='zh-CN')
            print("识别结果：", text)
            return text
        except Exception as e:
            print("语音识别失败:", e)
            return None

    def generate_report_with_llm(self, image_path, user_response, key_angles):
        prompt = (
            f"你是一个运动康复专家，请根据以下信息生成一份详细的运动姿势评估报告：\n"
            f"用户反馈：{user_response}\n"
            f"关键角度数据：\n"
            f"  髋关节角度：{key_angles['hip_angle']:.2f}\n"
            f"  膝关节角度：{key_angles['knee_angle']:.2f}\n"
            f"  踝关节角度：{key_angles['ankle_angle']:.2f}\n"
            f"请分析可能存在的问题，并给出改善建议和康复动作。"
        )
        
        qwen = OpenAI(
            # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
            api_key="sk-bbf9f24ff4194920a43e15749a2dad29",
            base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
        )
        # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models

        completion = qwen.chat.completions.create(
            model="qwen2.5-72b-instruct",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
        )
        generated_report = completion.choices[0].message.content
        print(generated_report)
        return generated_report

    def update_frame(self):
        ret, frame = self.cap.read()
        if not ret:
            return
        # 转为RGB并处理姿势检测
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.pose.process(frame_rgb)
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            left_shoulder = (landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value].y)
            left_hip = (landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value].y)
            left_knee = (landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                         landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE.value].y)
            left_ankle = (landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE.value].y)
            left_big_toe = (landmarks[self.mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,
                            landmarks[self.mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y)
            hip_angle = self.calculate_angle(left_shoulder, left_hip, left_knee)
            knee_angle = self.calculate_angle(left_hip, left_knee, left_ankle)
            ankle_angle = self.calculate_angle(left_knee, left_ankle, left_big_toe)
            hip_angle_complement = 180 - hip_angle
            knee_angle_complement = 180 - knee_angle
            ankle_angle_complement = 180 - ankle_angle

            # 在画面上显示角度信息
            cv2.putText(frame, f"hip: {hip_angle_complement:.2f}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"knee: {knee_angle_complement:.2f}", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"ankle: {ankle_angle_complement:.2f}", (20, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            cv2.line(frame, (int(left_shoulder[0] * frame.shape[1]), int(left_shoulder[1] * frame.shape[0])), 
                    (int(left_hip[0] * frame.shape[1]), int(left_hip[1] * frame.shape[0])), (0, 255, 0), 2)  # 连接肩膀和髋
            cv2.line(frame, (int(left_hip[0] * frame.shape[1]), int(left_hip[1] * frame.shape[0])), 
                    (int(left_knee[0] * frame.shape[1]), int(left_knee[1] * frame.shape[0])), (0, 255, 0), 2)  # 连接髋和膝
            cv2.line(frame, (int(left_knee[0] * frame.shape[1]), int(left_knee[1] * frame.shape[0])), 
                    (int(left_ankle[0] * frame.shape[1]), int(left_ankle[1] * frame.shape[0])), (0, 255, 0), 2)  # 连接膝和踝
            cv2.line(frame, (int(left_ankle[0] * frame.shape[1]), int(left_ankle[1] * frame.shape[0])), 
                    (int(left_big_toe[0] * frame.shape[1]), int(left_big_toe[1] * frame.shape[0])), (0, 255, 0), 2)  # 连接踝和大拇指


            cv2.circle(frame, (int(left_shoulder[0] * frame.shape[1]), int(left_shoulder[1] * frame.shape[0])), 5, (0, 0, 255), -1)  # 左肩
            cv2.circle(frame, (int(left_hip[0] * frame.shape[1]), int(left_hip[1] * frame.shape[0])), 5, (0, 255, 255), -1)  # 左髋
            cv2.circle(frame, (int(left_knee[0] * frame.shape[1]), int(left_knee[1] * frame.shape[0])), 5, (255, 0, 0), -1)  # 左膝
            cv2.circle(frame, (int(left_ankle[0] * frame.shape[1]), int(left_ankle[1] * frame.shape[0])), 5, (255, 255, 0), -1)  # 左踝
            cv2.circle(frame, (int(left_big_toe[0] * frame.shape[1]), int(left_big_toe[1] * frame.shape[0])), 5, (255, 0, 255), -1)  # 左脚大拇指

            # 满足条件时，提示受试者抬腿
            if (90 < ankle_angle_complement < 120) and (173 < knee_angle_complement < 180) and (175 < hip_angle_complement < 180):
                # self.speak("请抬左腿，保持膝盖打直和脚掌勾起，抬至最高点保持三秒")
                self.speak("请抬左腿")
                self.leg_raised = True

            if self.leg_raised:
                if hip_angle_complement < 150:
                    current_time = time.time()
                    if current_time - self.last_time >= 1:
                        self.last_time = current_time
                        if knee_angle_complement < 170:
                            self.speak("膝关节未伸直")
                        else:
                            if len(self.hip_angle_history) > 0:
                                hip_angle_diff = abs(hip_angle_complement - self.hip_angle_history[-1])
                            else:
                                hip_angle_diff = float('inf')
                            self.hip_angle_history.append(hip_angle_complement)
                            if len(self.hip_angle_history) > 3:
                                self.hip_angle_history.pop(0)
                            if len(self.hip_angle_history) > 2 and hip_angle_diff < 5:
                                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                                file_name = f"pose_{self.subject_id}_{timestamp}.jpg"
                                file_path = os.path.join(os.path.expanduser("~"), "Desktop", file_name)
                                cv2.putText(frame, f"Lift angle: {180-self.hip_angle_history[2]:.2f}", (20, 160),
                                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                                cv2.imwrite(file_path, frame)
                                self.speak("照片已拍摄并保存")
                                
                                self.speak("是否有麻木，牵扯，或疼痛？")
                                answer_1 =self.listen("是否有麻木，牵扯，或疼痛或其他症状？")

                                key_angles = {
                                    'hip_angle': hip_angle_complement,
                                    'knee_angle': knee_angle_complement,
                                    'ankle_angle': ankle_angle_complement
                                }
                                
                                report_content = self.generate_report_with_llm(file_path, answer_1, key_angles)

                                report_name = f"report_{self.subject_id}.docx"
                                report_path = os.path.join(os.path.expanduser("~"), "Desktop", report_name)
                                self.generate_report_with_image(file_path, report_content, report_path)
                                self.speak(f"报告已生成并保存至桌面")
                                self.timer.stop()
                                self.cap.release()
                                cv2.destroyAllWindows()
        
        # 将 frame 转为 QImage 显示到 QLabel 上
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        h, w, ch = frame_rgb.shape
        bytes_per_line = ch * w
        q_img = QImage(frame_rgb.data, w, h, bytes_per_line, QImage.Format.Format_RGB888)
        self.video_label.setPixmap(QPixmap.fromImage(q_img))
        
    def closeEvent(self, event):
        self.timer.stop()
        if self.cap is not None:
            self.cap.release()
        cv2.destroyAllWindows()
        event.accept()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = PoseApp()
    window.show()
    sys.exit(app.exec())

I0000 00:00:1744039437.890181 39431500 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1744039437.982430 39431641 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744039437.997859 39431634 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2025-04-07 23:23:58.963 python[11572:39431500] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-07 23:23:58.963 python[11572:39431500] +[IMKInputSession subclass]: chose IMKInputSession_Modern
2025-04-07 23:24:04.022 python[11572:39431500] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit
2025-04-07 23:24:06.686 python[11572:39431500] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Plea

请抬左腿
请抬左腿
照片已拍摄并保存
是否有麻木，牵扯，或疼痛？
是否有麻木，牵扯，或疼痛或其他症状？
请开始说话...
识别结果： 麻木
### 运动姿势评估报告

#### 用户基本信息
- **主要反馈**：麻木
- **关键角度数据**：
  - **髋关节角度**：120.53°
  - **膝关节角度**：177.01°
  - **踝关节角度**：135.40°

#### 分析与诊断

1. **髋关节角度（120.53°）**：
   - **正常范围**：站立时，髋关节角度通常在180°左右。
   - **分析**：髋关节角度较小，说明髋部可能存在屈曲状态，这可能导致腰部和臀部的肌肉紧张，进而影响神经分布，引起麻木感。

2. **膝关节角度（177.01°）**：
   - **正常范围**：站立时，膝关节角度接近180°。
   - **分析**：膝关节角度接近完全伸直，这可能是正常的，但如果伴随其他关节的问题，可能会导致下肢血液循环不畅，引发麻木。

3. **踝关节角度（135.40°）**：
   - **正常范围**：站立时，踝关节角度通常在90°左右。
   - **分析**：踝关节角度较大，说明脚踝处于过度背屈状态，这可能导致小腿肌肉紧张，影响血液循环，进一步加重麻木感。

#### 可能存在的问题
1. **髋部屈曲**：髋部屈曲可能导致腰部和臀部的肌肉紧张，影响坐骨神经的正常功能。
2. **踝关节过度背屈**：踝关节过度背屈可能导致小腿肌肉紧张，影响下肢血液循环，引起麻木。
3. **整体姿势不良**：综合上述关节的角度，用户的整体姿势可能存在前倾或后倾的问题，导致身体重心不稳，增加局部压力。

#### 改善建议
1. **改善髋部屈曲**：
   - **髋部拉伸**：每天进行髋部拉伸练习，如“鸽子式”瑜伽动作，每次保持30秒至1分钟。
   - **核心肌群训练**：加强腹部和背部的核心肌群，如平板支撑、桥式等，以改善整体姿态。

2. **调整踝关节角度**：
   - **小腿放松**：使用泡沫轴或按摩球放松小腿肌肉，每天10-15分钟。
   - **踝关节活动**：进行踝关节绕圈练习，每次10-15次，每天2-3组，以增强踝关节灵活性。

3. **整体姿势调整**：
   - **站姿训练**：保持正确的站

SystemExit: 0

/Users/zehaokou/miniconda3/envs/rehab/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


: 

In [2]:
import sys
import cv2
import mediapipe as mp
import math
import pyttsx3
import time
import os
from datetime import datetime
from docx import Document
from docx.shared import Inches
import threading
import queue

from PyQt5.QtWidgets import (QApplication, QMainWindow, QLabel, QLineEdit, QPushButton,
                             QVBoxLayout, QWidget, QMessageBox)
from PyQt5.QtCore import QTimer, Qt
from PyQt5.QtGui import QImage, QPixmap
from openai import OpenAI
import speech_recognition as sr


# Qwen API

In [3]:
qwen = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    api_key="sk-bbf9f24ff4194920a43e15749a2dad29",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)
# 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models

completion = qwen.chat.completions.create(
    model="qwen2.5-72b-instruct", 
    # model="deepseek-r1-distill-llama-70b",
    messages=[
        {'role': 'system', 'content': '你是个运动康复专家'},
        {'role': 'user', 'content': "你是个运动康复专家, 根据"},],
    )

output_test = completion.choices[0].message.content
print(output_test)

当然，作为运动康复专家，我可以帮助您解决与运动伤害、康复训练和预防措施相关的问题。请告诉我您具体需要咨询或了解的内容，比如受伤部位、目前的症状、想要达到的康复目标等，我会根据您的情况提供专业的建议和指导。如果您有任何疑问或者需要特定的帮助，请随时告诉我。


In [1]:
class PoseApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("姿势检测")
        self.subject_id = ""
        
        # UI控件：标签、输入框和按钮
        self.input_label = QLabel("请输入受试者编号:")
        self.subject_input = QLineEdit()
        self.start_button = QPushButton("开始检测")
        self.start_button.clicked.connect(self.start_detection)
        
        # 用于显示摄像头画面的标签
        self.video_label = QLabel()
        self.video_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        
        layout = QVBoxLayout()
        layout.addWidget(self.input_label)
        layout.addWidget(self.subject_input)
        layout.addWidget(self.start_button)
        layout.addWidget(self.video_label)
        
        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)
        
        # 初始化 mediapipe 和 pyttsx3
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose()
        self.mp_drawing = mp.solutions.drawing_utils
        self.engine = pyttsx3.init()
        
        # 建立专用语音播报队列和后台线程
        self.speech_queue = queue.Queue()
        self.speech_thread = threading.Thread(target=self.speech_loop, daemon=True)
        self.speech_thread.start()
        
        # 标记当前是否有语音在播报
        self.speech_in_progress = False
        
        # 摄像头和计时器
        self.cap = None
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_frame)
        
        # 用于逻辑判断的变量
        self.last_time = time.time()
        self.hip_angle_history = []
        self.leg_raised = False
        
        # 标志变量：确保“请抬左腿...”这句提示只在状态发生变化时播报
        self.left_leg_prompted = False

    def speech_loop(self):
        """后台线程循环读取语音队列并播报"""
        while True:
            text = self.speech_queue.get()
            if text is None:  # 收到退出信号
                break
            self.engine.say(text)
            self.engine.runAndWait()
            self.speech_queue.task_done()
            # 当前语音播报结束后，清除标记，让新语音有机会被播报
            self.speech_in_progress = False

    def speak(self, message):
        """
        若当前没有语音播报，则将消息加入队列开始播报；
        否则直接返回，不中断正在播放的语音。
        """
        if self.speech_in_progress:
            return
        self.speech_in_progress = True
        self.speech_queue.put(message)

    def calculate_angle(self, a, b, c):
        # 计算三点之间的夹角
        ab = [b[0] - a[0], b[1] - a[1]]
        bc = [c[0] - b[0], c[1] - b[1]]
        dot_product = ab[0] * bc[0] + ab[1] * bc[1]
        cross_product = ab[0] * bc[1] - ab[1] * bc[0]
        angle = math.degrees(math.atan2(abs(cross_product), dot_product))
        return angle

    def ask_question(self, question):
        # 使用消息对话框进行简单的“是/否”提问
        msg_box = QMessageBox(self)
        msg_box.setWindowTitle("提问")
        msg_box.setText(question + "\n点击 [是] 或 [否]")
        yes_button = msg_box.addButton("是", QMessageBox.ButtonRole.YesRole)
        no_button = msg_box.addButton("否", QMessageBox.ButtonRole.NoRole)
        msg_box.exec()
        if msg_box.clickedButton() == yes_button:
            return "是"
        else:
            return "否"

    def generate_report_with_image(self, image_path, report_content, report_path):
        doc = Document()
        doc.add_heading('姿势检测报告', 0)
        doc.add_paragraph(f"报告生成时间：{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        # 创建表格
        table = doc.add_table(rows=1, cols=2)
        table.style = 'Table Grid'
        hdr_cells = table.rows[0].cells
        hdr_cells[0].text = '症状'
        hdr_cells[1].text = '描述'
        for cell in hdr_cells:
            cell.width = Inches(3)
        symptoms = report_content.split('；')
        for symptom in symptoms:
            row_cells = table.add_row().cells
            symptom_parts = symptom.split('：')
            if len(symptom_parts) == 2:
                row_cells[0].text = symptom_parts[0]
                row_cells[1].text = symptom_parts[1]
        doc.add_paragraph("照片：")
        doc.add_picture(image_path, width=Inches(3))
        doc.save(report_path)

    def start_detection(self):
        # 获取受试者编号，并启动摄像头检测
        self.subject_id = self.subject_input.text().strip()
        if not self.subject_id:
            QMessageBox.warning(self, "警告", "请输入受试者编号!")
            return
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            QMessageBox.critical(self, "错误", "无法打开摄像头!")
            return
        self.timer.start(30)  # 每30毫秒更新一次画面
        self.start_button.setEnabled(False)
        self.subject_input.setEnabled(False)

    def listen(prompt):
        print(prompt)  
        r = sr.Recognizer()
        with sr.Microphone() as source:
            print("请开始说话...")
            audio = r.listen(source)
        try:
            text = r.recognize_google(audio, language='zh-CN')
            print("识别结果：", text)
            return text
        except Exception as e:
            print("语音识别失败:", e)
            return None

    def generate_report_with_llm(image_path, user_response, key_angles):
        prompt = (
            f"你是一个运动康复专家，请根据以下信息生成一份详细的运动姿势评估报告：\n"
            f"用户反馈：{user_response}\n"
            f"关键角度数据：\n"
            f"  髋关节角度：{key_angles['hip_angle']:.2f}\n"
            f"  膝关节角度：{key_angles['knee_angle']:.2f}\n"
            f"  踝关节角度：{key_angles['ankle_angle']:.2f}\n"
            f"请分析可能存在的问题，并给出改善建议和康复动作。"
        )
        
        completion = qwen.chat.completions.create(
            model="qwen2.5-72b-instruct",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
        )
        generated_report = completion.choices[0].message.content
        return generated_report


    def update_frame(self):
        ret, frame = self.cap.read()
        if not ret:
            return
        # 转为RGB并处理姿势检测
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.pose.process(frame_rgb)
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            left_shoulder = (landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value].y)
            left_hip = (landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value].y)
            left_knee = (landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                         landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE.value].y)
            left_ankle = (landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE.value].y)
            left_big_toe = (landmarks[self.mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,
                            landmarks[self.mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y)
            hip_angle = self.calculate_angle(left_shoulder, left_hip, left_knee)
            knee_angle = self.calculate_angle(left_hip, left_knee, left_ankle)
            ankle_angle = self.calculate_angle(left_knee, left_ankle, left_big_toe)
            hip_angle_complement = 180 - hip_angle
            knee_angle_complement = 180 - knee_angle
            ankle_angle_complement = 180 - ankle_angle

            # 在画面上显示角度信息
            cv2.putText(frame, f"hip: {hip_angle_complement:.2f}", (20, 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"knee: {knee_angle_complement:.2f}", (20, 80), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"ankle: {ankle_angle_complement:.2f}", (20, 120), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            cv2.line(frame, (int(left_shoulder[0] * frame.shape[1]), int(left_shoulder[1] * frame.shape[0])), 
                     (int(left_hip[0] * frame.shape[1]), int(left_hip[1] * frame.shape[0])), (0, 255, 0), 2)
            cv2.line(frame, (int(left_hip[0] * frame.shape[1]), int(left_hip[1] * frame.shape[0])), 
                     (int(left_knee[0] * frame.shape[1]), int(left_knee[1] * frame.shape[0])), (0, 255, 0), 2)
            cv2.line(frame, (int(left_knee[0] * frame.shape[1]), int(left_knee[1] * frame.shape[0])), 
                     (int(left_ankle[0] * frame.shape[1]), int(left_ankle[1] * frame.shape[0])), (0, 255, 0), 2)
            cv2.line(frame, (int(left_ankle[0] * frame.shape[1]), int(left_ankle[1] * frame.shape[0])), 
                     (int(left_big_toe[0] * frame.shape[1]), int(left_big_toe[1] * frame.shape[0])), (0, 255, 0), 2)

            cv2.circle(frame, (int(left_shoulder[0] * frame.shape[1]), int(left_shoulder[1] * frame.shape[0])), 5, (0, 0, 255), -1)
            cv2.circle(frame, (int(left_hip[0] * frame.shape[1]), int(left_hip[1] * frame.shape[0])), 5, (0, 255, 255), -1)
            cv2.circle(frame, (int(left_knee[0] * frame.shape[1]), int(left_knee[1] * frame.shape[0])), 5, (255, 0, 0), -1)
            cv2.circle(frame, (int(left_ankle[0] * frame.shape[1]), int(left_ankle[1] * frame.shape[0])), 5, (255, 255, 0), -1)
            cv2.circle(frame, (int(left_big_toe[0] * frame.shape[1]), int(left_big_toe[1] * frame.shape[0])), 5, (255, 0, 255), -1)

            # 判断左腿抬起条件
            condition = (100 < ankle_angle_complement < 110) and \
                        (173 < knee_angle_complement < 180) and \
                        (175 < hip_angle_complement < 180)
            if condition:
                # 只有当条件刚满足且当前没有语音在播报时，才播报提示语
                if not self.left_leg_prompted and not self.speech_in_progress:
                    self.speak("请抬左腿，保持膝盖打直和脚掌勾起，抬至最高点保持三秒")
                    self.left_leg_prompted = True
                self.leg_raised = True
            else:
                # 重置提示标志，当条件不满足时
                self.left_leg_prompted = False

            # 如果左腿抬起状态进入后续判断逻辑
            if self.leg_raised:
                if hip_angle_complement < 150:
                    current_time = time.time()
                    if current_time - self.last_time >= 1:
                        self.last_time = current_time
                        if knee_angle_complement < 170 and not self.speech_in_progress:
                            self.speak("膝关节未伸直")
                        else:
                            if len(self.hip_angle_history) > 0:
                                hip_angle_diff = abs(hip_angle_complement - self.hip_angle_history[-1])
                            else:
                                hip_angle_diff = float('inf')
                            self.hip_angle_history.append(hip_angle_complement)
                            if len(self.hip_angle_history) > 3:
                                self.hip_angle_history.pop(0)
                            if len(self.hip_angle_history) > 2 and hip_angle_diff < 5:
                                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                                file_name = f"pose_{self.subject_id}_{timestamp}.jpg"
                                file_path = os.path.join(os.path.expanduser("~"), "Desktop", file_name)
                                cv2.putText(frame, f"Lift angle: {180-self.hip_angle_history[2]:.2f}", 
                                            (20, 160), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                                cv2.imwrite(file_path, frame)
                                self.speak("照片已拍摄并保存")
                                
                                # self.speak("是否有麻木，牵扯，或疼痛？")
                                # answer_1 = self.ask_question("是否麻木？")
                                self.speak("是否有麻木，牵扯，或疼痛？")
                                answer_1 =self.listen("是否有麻木，牵扯，或疼痛？")

                                key_angles = {
                                    'hip_angle': hip_angle_complement,
                                    'knee_angle': knee_angle_complement,
                                    'ankle_angle': ankle_angle_complement
                                }
                                
                                report_content = self.generate_report_with_llm(file_path, answer_1, key_angles)

                                report_name = f"report_{self.subject_id}.docx"
                                report_path = os.path.join(os.path.expanduser("~"), "Desktop", report_name)
                                self.generate_report_with_image(file_path, report_content, report_path)
                                self.speak(f"报告已生成并保存至桌面")
                                self.timer.stop()
                                self.cap.release()
                                cv2.destroyAllWindows()
        
        # 将 frame 转为 QImage 显示到 QLabel 上
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        h, w, ch = frame_rgb.shape
        bytes_per_line = ch * w
        q_img = QImage(frame_rgb.data, w, h, bytes_per_line, QImage.Format.Format_RGB888)
        self.video_label.setPixmap(QPixmap.fromImage(q_img))
        
    def closeEvent(self, event):
        self.timer.stop()
        if self.cap is not None:
            self.cap.release()
        cv2.destroyAllWindows()
        # 通知语音线程退出
        self.speech_queue.put(None)
        event.accept()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = PoseApp()
    window.show()
    sys.exit(app.exec())

NameError: name 'QMainWindow' is not defined